In [33]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# 'cs231n' folder containing the '.py', 'classifiers' and 'datasets'
# folders.
# e.g. 'cs231n/assignments/assignment1/cs231n/'
FOLDERNAME = 'cs231n/assignments/assignment1/cs231n/'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../
%cd ../../
%cd cs231n/datasets/
!bash get_datasets.sh
%cd ../../

Mounted at /content/drive
/content/drive/My Drive
/content
/content/cs231n/datasets
--2020-08-14 08:29:21--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  31.2MB/s    in 5.8s    

2020-08-14 08:29:27 (27.8 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
/content


# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [34]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [35]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for your own interest.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [36]:
from cs231n.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

Done extracting features for 1000 / 49000 images
Done extracting features for 2000 / 49000 images
Done extracting features for 3000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 5000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 7000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 9000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 11000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 13000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 15000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 17000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 19000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extr

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [37]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [5e4, 5e5, 5e6]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****



# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
import copy
lrs=np.geomspace(1e-9, 5e-6, num=6)
rgs=np.geomspace(1e4, 5e7, num=6)
A=np.transpose([np.tile(lrs, len(rgs)), np.repeat(rgs, len(lrs))])



for i in range(A.shape[0]):
    lr=A[i][0]
    rg=A[i][1]
    print(i)
    svm = LinearSVM()
    svm.train(X_train_feats, y_train, learning_rate=lr, reg=rg, num_iters=1500, verbose=False)

    y_train_pred = svm.predict(X_train_feats)
    y_val_pred = svm.predict(X_val_feats)
    ta=np.mean(y_train == y_train_pred)
    va=np.mean(y_val == y_val_pred)
    inp=(lr,rg)
    opt=(ta,va)

    results[lr, rg] = [ta, va]

    if va>best_val:
      best_val=va
      best_svm = svm

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
lr 1.000000e-09 reg 1.000000e+04 train accuracy: 0.086347 val accuracy: 0.078000
lr 1.000000e-09 reg 5.492803e+04 train accuracy: 0.104224 val accuracy: 0.097000
lr 1.000000e-09 reg 3.017088e+05 train accuracy: 0.107347 val accuracy: 0.088000
lr 1.000000e-09 reg 1.657227e+06 train accuracy: 0.103143 val accuracy: 0.115000
lr 1.000000e-09 reg 9.102821e+06 train accuracy: 0.103939 val accuracy: 0.109000
lr 1.000000e-09 reg 5.000000e+07 train accuracy: 0.094755 val accuracy: 0.092000
lr 5.492803e-09 reg 1.000000e+04 train accuracy: 0.081184 val accuracy: 0.092000
lr 5.492803e-09 reg 5.492803e+04 train accuracy: 0.092816 val accuracy: 0.111000
lr 5.492803e-09 reg 3.017088e+05 train accuracy: 0.114102 val accuracy: 0.122000
lr 5.492803e-09 reg 1.657227e+06 train accuracy: 0.101571 val accuracy: 0.101000
lr 5.492803e-09 reg 9.102821e+06 train accuracy: 0.110714 val accuracy: 0.108000
lr 5.492803

In [38]:
# Evaluate your trained SVM on the test set: you should be able to get at least 0.40
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

0.416


In [39]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### Inline question 1:
Describe the misclassification results that you see. Do they make sense?


$\color{blue}{\textit Your Answer:}$Since svm makes templates for image classes and since a huge part of each picture is its backgrounds, our svm classifier is likely to only judge based on the background.
For example, consider plane class. Since most airplane photos are captured while planes are flying, they havem ostly blue backgrounds because of the presence of sky. So, other pictrues that have blue backgrounds are likely to be misclassifed. Like a horse near the sea which sea fools our svm classifier to think it's a plane




## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [40]:
# Preprocessing: Remove the bias dimension
# Make sure to run this cell only ONCE
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

(49000, 155)
(49000, 154)


In [49]:
from cs231n.classifiers.neural_net import TwoLayerNet

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
import copy
results={}
lrs=np.geomspace(1e-4, 5e-1, num=8)
rgs=np.geomspace(1e-7, 1e1, num=7)
A=np.transpose([np.tile(lrs, len(rgs)), np.repeat(rgs, len(lrs))])

best=None
best_val=-1


for i in range (A.shape[0]):
  lr=A[i][0]
  rg=A[1][1]
  
  input_size = 154
  hidden_size = 50
  num_classes = 10
  net = TwoLayerNet(input_size, hidden_size, num_classes)

  # Train the network
  stats = net.train(X_train_feats, y_train, X_val_feats, y_val,num_iters=1000, batch_size=200,learning_rate=lr, learning_rate_decay=0.95,reg=rg, verbose=False)

  # Predict on the validation set
  val_acc = (net.predict(X_val_feats) == y_val).mean()
  print('Validation accuracy: ', val_acc)

  ta=(net.predict(X_train_feats) ==y_train).mean()
  va=val_acc
  inp=(lr,rg)
  opt=(ta,va)

  print(i)
  results[inp]=opt
  

  if va>best_val:
    #print("IN")
    best_val=va
    best=inp
    best_net=net
   


pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


Validation accuracy:  0.102
0
Validation accuracy:  0.087
1
Validation accuracy:  0.098
2
Validation accuracy:  0.098
3
Validation accuracy:  0.112
4
Validation accuracy:  0.359
5
Validation accuracy:  0.526
6
Validation accuracy:  0.547
7
Validation accuracy:  0.078
8
Validation accuracy:  0.087
9
Validation accuracy:  0.078
10
Validation accuracy:  0.107
11
Validation accuracy:  0.087
12
Validation accuracy:  0.382
13
Validation accuracy:  0.532
14
Validation accuracy:  0.521
15
Validation accuracy:  0.105
16
Validation accuracy:  0.113
17
Validation accuracy:  0.078
18
Validation accuracy:  0.102
19
Validation accuracy:  0.087
20
Validation accuracy:  0.351
21
Validation accuracy:  0.518
22
Validation accuracy:  0.545
23
Validation accuracy:  0.078
24
Validation accuracy:  0.079
25
Validation accuracy:  0.078
26
Validation accuracy:  0.079
27
Validation accuracy:  0.099
28
Validation accuracy:  0.374
29
Validation accuracy:  0.509
30
Validation accuracy:  0.556
31
Validation accurac

In [53]:
# Run your best neural net classifier on the test set. You should be able
# to get more than 55% accuracy.

input_size = 154
hidden_size = 50
num_classes = 10
best_net = TwoLayerNet(input_size, hidden_size, num_classes)

 # Train the network
stats = best_net.train(X_train_feats, y_train, X_val_feats, y_val,num_iters=5000, batch_size=1000,learning_rate=best[0], learning_rate_decay=0.95,reg=best[1], verbose=True)
test_acc = (best_net.predict(X_test_feats) == y_test).mean()
print(test_acc)

iteration 0 / 5000: loss 2.302585
iteration 100 / 5000: loss 1.615779
iteration 200 / 5000: loss 1.429758
iteration 300 / 5000: loss 1.304984
iteration 400 / 5000: loss 1.281951
iteration 500 / 5000: loss 1.278247
iteration 600 / 5000: loss 1.233866
iteration 700 / 5000: loss 1.165461
iteration 800 / 5000: loss 1.208813
iteration 900 / 5000: loss 1.228565
iteration 1000 / 5000: loss 1.132299
iteration 1100 / 5000: loss 1.132765
iteration 1200 / 5000: loss 1.130774
iteration 1300 / 5000: loss 1.139707
iteration 1400 / 5000: loss 1.033130
iteration 1500 / 5000: loss 1.162196
iteration 1600 / 5000: loss 1.112586
iteration 1700 / 5000: loss 1.084296
iteration 1800 / 5000: loss 1.114627
iteration 1900 / 5000: loss 1.094093
iteration 2000 / 5000: loss 1.147048
iteration 2100 / 5000: loss 1.083201
iteration 2200 / 5000: loss 1.105247
iteration 2300 / 5000: loss 1.065692
iteration 2400 / 5000: loss 1.087830
iteration 2500 / 5000: loss 1.059800
iteration 2600 / 5000: loss 1.055384
iteration 270

---
# IMPORTANT

This is the end of this question. Please do the following:

1. Click `File -> Save` to make sure the latest checkpoint of this notebook is saved to your Drive.
2. Execute the cell below to download the modified `.py` files back to your drive.

In [54]:
import os

FOLDER_TO_SAVE = os.path.join('drive/My Drive/', FOLDERNAME)
FILES_TO_SAVE = []

for files in FILES_TO_SAVE:
  with open(os.path.join(FOLDER_TO_SAVE, '/'.join(files.split('/')[1:])), 'w') as f:
    f.write(''.join(open(files).readlines()))